In [1]:
#@title LICENSE

# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Vertex AI Extensions with API Key Authentication

## Overview

Vertex AI Extensions is a platform for creating and managing extensions that connect large language models to external systems via APIs. These external systems can provide LLMs with real-time data and perform data processing actions on their behalf. You can use pre-built or third-party extensions in Vertex AI Extensions.

Learn more about [Vertex AI Extensions](https://cloud.google.com/vertex-ai/docs/generative-ai/extensions/private/overview).

This notebook provides guidance on how to build a Vertex AI Extension that requires authentication to an external service. This guide assumes that you are familiar with the Vertex AI Python SDK, [OpenAPI specification](https://swagger.io/specification/), the [Google Maps Places API](https://developers.google.com/maps/documentation/places/web-service/search), and [Cloud Secret Manager](https://cloud.google.com/secret-manager/docs).

### Objective

In this tutorial, you'll learn how to create a secret on Secret Manager, create an extension service backend on Cloud Run, and register the extension with Vertex AI Extensions.

The steps performed include:

- Storing a secret on Cloud Secret Manager
- Use Google Maps as an extension tool
- Creating an OpenAPI 3.1 YAML file for the external services
- Registering the extension with Vertex AI
- Verify the behavior of the extension

### Additional information

This tutorial uses the following Google services and resources:

- Vertex AI Extensions
- Cloud API Key API
- Cloud Secret Manager
- Google Maps Places API

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

### Authenticate your Google Cloud account

You must authenticate to Google Cloud to access the pre-release version of the Python SDK and the Vertex AI Extensions feature.

In [7]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth
    auth.authenticate_user()

### Installation

This tutorial requires a pre-release version of the Python SDK for Vertex AI. You must be logged in with credentials that are registered for the Vertex AI Extensions Private Preview.

Run the following command to download the library as a wheel from a Cloud Storage bucket:

In [2]:
!gsutil cp gs://vertex_sdk_private_releases/llm_extension/google_cloud_aiplatform-1.39.dev20231219+llm.extension-py2.py3-none-any.whl .

Then, install the following packages required to execute this notebook:

In [3]:
!pip install --force-reinstall --quiet google_cloud_aiplatform-1.39.dev20231219+llm.extension-py2.py3-none-any.whl
!pip install --upgrade --quiet google-cloud-api-keys \
google-cloud-iam \
google-cloud-resource-manager \
google-cloud-secret-manager \
google-cloud-storage \
"shapely<2"

Restart the kernel after installing packages:

In [4]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

Install all other requirements.

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Your project must also be allowlisted for the Vertex AI Extension Private Preview.
1. This notebook requires that you have the following permissions for your GCP project:
- `roles/aiplatform.user`

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [5]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}

### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [6]:
REGION = "us-central1"  # @param {type: "string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [8]:
BUCKET_NAME = "your-bucket-name"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
extensions_prefix = "extension"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [9]:
!gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://vertex-ai-extensions-koverholt/...
ServiceException: 409 A Cloud Storage bucket named 'vertex-ai-extensions-koverholt' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Import libraries



In [10]:
import base64
import os

import vertexai
from google.cloud.aiplatform.private_preview import llm_extension

from google.cloud import api_keys_v2
from google.cloud import resourcemanager_v3
from google.cloud import secretmanager
from google.cloud import storage

from google.iam import v1 as iam

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [11]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Create and store an API key

Working with HTTP authentication that requires an API key, the Vertex AI Extensions framework integrates with Secret Manager for secret storage and access. The Vertex AI Extension platform does not store the secret data directly; instead, users must manage the lifecycle of their Secret Manager resources.

### Create an API key for use with Google Maps

To demonstrate how to use an API key with Vertex AI Extensions, this notebook shows how to call into the Google Maps API. You must create an API key to authenticate with the Google Maps API.

You can learn more about how to get an API key for Google Maps on [this page](https://developers.google.com/maps/documentation/places/web-service/get-api-key).

In [12]:
# Log into your Google Cloud account.
# Follow the directions printed to the cell output.
!gcloud auth application-default login

# Set the project ID
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [13]:
KEY_ID = "vertex-extensions"

api_key_client = api_keys_v2.ApiKeysClient()
key = api_keys_v2.Key()
key.display_name = "My key for Vertex Extensions"

api_target = api_keys_v2.ApiTarget()
api_target.service = "places-backend.googleapis.com"

restrictions = api_keys_v2.Restrictions()
restrictions.api_targets = [api_target]
key.restrictions = restrictions

request = api_keys_v2.CreateKeyRequest()
request.parent = f"projects/{PROJECT_ID}/locations/global"
request.key = key

key_response = api_key_client.create_key(request=request).result()
print(f"API key resource name: {key_response.name}")

API key resource name: projects/964731510884/locations/global/keys/4caf1e5d-01e1-4c2d-866f-b2846736f5bb


Note that it is not a best practice to print API keys in a production environment.

In [14]:
api_key_string = key_response.key_string

### Store the API key in Secret Manager

Now that you have the API key, you can store it in Secret Manager for use with Vertex AI Extensions

In [15]:
SECRET_ID = "vertex-extensions-secret"

secret_manager_client = secretmanager.SecretManagerServiceClient()
parent = f"projects/{PROJECT_ID}"

secret = secret_manager_client.create_secret(
    request={
        "parent": parent,
        "secret_id": SECRET_ID,
        "secret": {"replication": {"automatic": {}}},
    }
)

In [16]:
api_key_bytes = api_key_string.encode("ascii")

version = secret_manager_client.add_secret_version(
    request={"parent": secret.name, "payload": {"data": api_key_bytes}}
)

In [17]:
# Verify that you can access the secret.
response = secret_manager_client.access_secret_version(request={"name": version.name})
stored_b64_key = response.payload.data
round_tripped_key = stored_b64_key.decode('ascii')

assert api_key_string == round_tripped_key

### Allow the Vertex AI service account to access Secret Manager

Now that you have your API key stored in Secret Manager, you must permit Vertex AI to access your secret. To do this, you must assign the Secret Manager Secret Accessor role, `roles/secretmanager.secretAccessor`, to the service account used by Vertex AI. The service account used by Vertex AI usually has an ID in the following format:

```
service-[PROJECT_NUMBER]@gcp-sa-aiplatform.iam.gserviceaccount.com
```

<div style="background-color:rgb(235,186,52); padding:5px; color:rgb(105, 83, 23);"><strong>WARNING:</strong> The following cells use the <span style="font-family:'Courier'">resourcemanager.ProjectsClient.set_iam_policy()</span> to grant a role to a service account. The <span style="font-family:'Courier'">set_iam_policy()</span> method can potentially delete all of the bindings in your Google Cloud project's policy. If you prefer to use the Google Cloud console, refer to the <a href="https://cloud.google.com/iam/docs/granting-changing-revoking-access#grant-single-role" target="_blank">documentation</a>.</div>

In [18]:
project_client = resourcemanager_v3.ProjectsClient()
request = resourcemanager_v3.GetProjectRequest(name=f"projects/{PROJECT_ID}")

# Make the request
project = project_client.get_project(request=request)

In [19]:
project_number = project.name.split("/")[-1]
vertex_sa = f"service-{project_number}@gcp-sa-aiplatform.iam.gserviceaccount.com"

In [20]:
policy = project_client.get_iam_policy(request={
    "resource": f"projects/{PROJECT_ID}"
})

role = "roles/secretmanager.secretAccessor"
bindings = list(b for b in policy.bindings if b.role == role)

# Check to see whether this policy binding already exists
if len(bindings) == 0:
    binding = iam.policy_pb2.Binding(role=role, members=[vertex_sa])
    policy.bindings.append(binding)

else:
    binding = bindings[0]
    binding.members.append(vertex_sa)

print(policy)

**BEFORE RUNNING THE NEXT CELL:** Review the output in the previous cell to ensure that no critical bindings are lost. Successfully running the next cell sets the policy on the Google Project to exactly what is specified in the `policy` object.

Note that IAM restrictions on your user account or Compute service account might not allow you to set the IAM policy programmatically. If you cannot set the policy programmatically, try [setting the policy in the Cloud Console](https://cloud.google.com/iam/docs/granting-changing-revoking-access#grant-single-role).

In [21]:
updated_policy = project_client.set_iam_policy({
    "resource": f"projects/{PROJECT_ID}",
    "policy": policy,
})

print(updated_policy)

## Create a Google Maps extension

In this tutorial, you create a simple Vertex AI extension that calls the Google Maps Places API. This service returns the place information from a given text prompt. You can read the [Google Maps Places API documentation](https://developers.google.com/maps/documentation/places/web-service/search) for more details.

### Upload the OpenAPI YAML file

Your Vertex Extension requires an OpenAPI 3.1 YAML file that defines routes, URL, HTTP methods, requests, and responses from the Places API. The following code creates a YAML file that you need to upload to your Cloud Storage bucket.

In [22]:
if not os.path.exists("extension-api"):
    os.mkdir("extension-api")

openapi_yaml = f"""
openapi: "3.1.0"
info:
  version: 1.0.0
  title: google_map_tool
  description: This tool uses Google Map to do text search
servers:
  - url: https://maps.googleapis.com
paths:
  /maps/api/place/textsearch/json:
    get:
      operationId: textSearch
      description: The Google Places API Text Search Service is a web service that returns information about a set of places based on a string — for example "pizza in New York" or "shoe stores near Ottawa" or "123 Main Street". The service responds with a list of places matching the text string and any location bias that has been set.
      parameters:
        - name: query
          description: |
           The text string on which to search, for example: "restaurant" or "123 Main Street". This must be a place name, address, or category of establishments.
          schema:
            type: string
          required: true
          in: query
      responses:
          '200':
            description: List of one or more places on a map.
            content:
              application/json:
                schema:
                  $ref: "#/components/schemas/Place"
components:
  schemas:
    Place:
      description: Represents a place.
      properties:
        name:
          type: string
        description:
          type: string
        rating:
          type: string
        url:
          type: string
          nullable: true
        review_count:
          type: number
        address:
          type: string
"""

print(openapi_yaml)


openapi: "3.1.0"
info:
  version: 1.0.0
  title: google_map_tool
  description: This tool uses Google Map to do text search
servers:
  - url: https://maps.googleapis.com
paths:
  /maps/api/place/textsearch/json:
    get:
      operationId: textSearch
      description: The Google Places API Text Search Service is a web service that returns information about a set of places based on a string — for example "pizza in New York" or "shoe stores near Ottawa" or "123 Main Street". The service responds with a list of places matching the text string and any location bias that has been set.
      parameters:
        - name: query
          description: |
           The text string on which to search, for example: "restaurant" or "123 Main Street". This must be a place name, address, or category of establishments.
          schema:
            type: string
          required: true
          in: query
      responses:
          '200':
            description: List of one or more places on a map.


In [23]:
%store openapi_yaml >extension-api/maps-extension.yaml

Writing 'openapi_yaml' (str) to file 'extension-api/maps-extension.yaml'.


Upload the OpenAPI YAML to your Cloud Storage bucket.

In [24]:
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob_name = f"{extensions_prefix}/maps-extension.yaml"
blob = bucket.blob(blob_name)
blob.upload_from_filename("extension-api/maps-extension.yaml")

### Create the extension

 following cell uses the Python SDK to import the extension (thereby creating it) in Vertex AI.

In [25]:
extension_maps = llm_extension.Extension.create(
    display_name = "Google Maps Places API",
    description = "Google Maps Places API Extension",
    manifest = {
        "name": "google_maps_tool",
        "description": "This extension calls the Google Maps Places API to get information",
        "api_spec": {
            "open_api_gcs_uri": f"gs://{BUCKET_NAME}/{extensions_prefix}/maps-extension.yaml"
        },
        "auth_config": {
            "auth_type": "API_KEY_AUTH",
            "api_key_config": {
                "name": "key",
                "api_key_secret": version.name,
                "http_element_location": "HTTP_IN_QUERY",
            },
        },
    },
)
extension_maps

Creating Extension
Create Extension backing LRO: projects/964731510884/locations/us-central1/extensions/1910370666935222272/operations/665970825689563136
Extension created. Resource name: projects/964731510884/locations/us-central1/extensions/1910370666935222272
To use this Extension in another session:
extension = aiplatform.Extension('projects/964731510884/locations/us-central1/extensions/1910370666935222272')


resource name: projects/964731510884/locations/us-central1/extensions/1910370666935222272

Now that you've create your extension, let's confirm that it's registered:

In [26]:
print("Name:", extension_maps.gca_resource.name)
print("Display Name:", extension_maps.display_name)
print("Description:", extension_maps.gca_resource.description)

Name: projects/964731510884/locations/us-central1/extensions/1910370666935222272
Display Name: Google Maps Places API
Description: Google Maps Places API Extension


### Run the Vertex extension

Once you have created a Vertex AI extension, you can execute the extension to verify its behavior.

In [27]:
response = extension_maps.execute(
    "textSearch",
    operation_params = {
        "query": "1155 Borregas Ave",
    },
)
response

{'html_attributions': [],
 'results': [{'formatted_address': 'Google Building MP1, 1155 Borregas Ave, Sunnyvale, CA 94089, USA',
   'geometry': {'location': {'lat': 37.404937, 'lng': -122.0214239},
    'viewport': {'northeast': {'lat': 37.40641357989272,
      'lng': -122.0199238701073},
     'southwest': {'lat': 37.40371392010728, 'lng': -122.0226235298927}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
   'icon_background_color': '#7B9EB0',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
   'name': 'Google Building MP1',
   'photos': [{'height': 3024,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100557233042623135184">Elmer Garduno</a>'],
     'photo_reference': 'AWU5eFikh3tZcNFvSRY-YiybUfviwGCGszLFkThMI0r_gA3z455ZAXuF56eIx_pP4O33WWUuOiBWhcseAV6kmoDDnznboHXrIFJaAC8Wxj3jkrvEZM89fxAMCWUZGxGkNISOnn9Mjnw6vF9yXd1vnTNZ1-4YsqARwp8cvFXYvDIr4rGMEO1Z',
     'width': 4032}],

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

In [28]:
# Delete the extension
extension_maps.delete()

# Delete Cloud Storage objects that were created
# delete_bucket = False
# if delete_bucket or os.getenv("IS_TESTING"):
# ! gsutil -m rm -r $BUCKET_URI

Deleting Extension : projects/964731510884/locations/us-central1/extensions/1910370666935222272
Delete Extension  backing LRO: projects/964731510884/locations/us-central1/operations/4561021553411620864
Extension deleted. . Resource name: projects/964731510884/locations/us-central1/extensions/1910370666935222272
